In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random
import Sastrawi
import pandas as pd

import keras

Using TensorFlow backend.


In [2]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)
    print intents



{u'intents': [{u'patterns': [u'Hi', u'How are you', u'Is anyone there?', u'Hello', u'Good day'], u'tag': u'greeting', u'responses': [u'Hello, thanks for visiting', u'Good to see you again', u'Hi there, how can I help?'], u'context_set': u''}, {u'patterns': [u'Bye', u'See you later', u'Goodbye'], u'tag': u'goodbye', u'responses': [u'See you later, thanks for visiting', u'Have a nice day', u'Bye! Come back again soon.']}, {u'patterns': [u'Thanks', u'Thank you', u"That's helpful"], u'tag': u'thanks', u'responses': [u'Happy to help!', u'Any time!', u'My pleasure']}, {u'patterns': [u'What hours are you open?', u'What are your hours?', u'When are you open?'], u'tag': u'hours', u'responses': [u"We're open every day 9am-9pm", u'Our hours are 9am-9pm every day']}, {u'patterns': [u'Which mopeds do you have?', u'What kinds of mopeds are there?', u'What do you rent?'], u'tag': u'mopeds', u'responses': [u'We rent Yamaha, Piaggio and Vespa mopeds', u'We have Piaggio, Vespa and Yamaha mopeds']}, {u'p

In [5]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

#print (len(documents), "documents")
#print (len(classes), "classes", classes)
#print (len(words), "unique stemmed words", words)

print documents

[([u'Hi'], u'greeting'), ([u'How', u'are', u'you'], u'greeting'), ([u'Is', u'anyone', u'there', u'?'], u'greeting'), ([u'Hello'], u'greeting'), ([u'Good', u'day'], u'greeting'), ([u'Bye'], u'goodbye'), ([u'See', u'you', u'later'], u'goodbye'), ([u'Goodbye'], u'goodbye'), ([u'Thanks'], u'thanks'), ([u'Thank', u'you'], u'thanks'), ([u'That', u"'s", u'helpful'], u'thanks'), ([u'What', u'hours', u'are', u'you', u'open', u'?'], u'hours'), ([u'What', u'are', u'your', u'hours', u'?'], u'hours'), ([u'When', u'are', u'you', u'open', u'?'], u'hours'), ([u'Which', u'mopeds', u'do', u'you', u'have', u'?'], u'mopeds'), ([u'What', u'kinds', u'of', u'mopeds', u'are', u'there', u'?'], u'mopeds'), ([u'What', u'do', u'you', u'rent', u'?'], u'mopeds'), ([u'Do', u'you', u'take', u'credit', u'cards', u'?'], u'payments'), ([u'Do', u'you', u'accept', u'Mastercard', u'?'], u'payments'), ([u'Are', u'you', u'cash', u'only', u'?'], u'payments'), ([u'Are', u'you', u'open', u'today', u'?'], u'opentoday'), ([u'When

In [9]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

len(train_x)

27

In [11]:
# reset underlying graph data
tf.reset_default_graph()
keras.
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.59983 | time: 0.016s
| Adam | epoch: 1000 | loss: 0.59983 - acc: 0.9273 -- iter: 24/27
Training Step: 4000  | total loss: 0.59306 | time: 0.023s
| Adam | epoch: 1000 | loss: 0.59306 - acc: 0.9013 -- iter: 27/27
--
INFO:tensorflow:/home/ahmad/Documents/belajar/chatbot-practice-master/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [13]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [15]:
p = bow("is your shop open today?", words)
print (p)
print (classes)
print(model.predict([p]))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
[u'goodbye', u'greeting', u'hours', u'mopeds', u'opentoday', u'payments', u'rental', u'thanks', u'today']
[[  1.27641752e-03   2.55473959e-03   5.65188378e-02   2.64283171e-05
    5.29385924e-01   1.10234844e-03   4.45087535e-05   2.88776070e-01
    1.20314747e-01]]


In [16]:
# save all of our data structures
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

In [17]:
model.load('./model.tflearn')

In [21]:
# create a data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return random.choice(i['responses'])

            results.pop(0)

In [22]:
classify('is your shop open today?')

[(u'opentoday', 0.88606346)]

In [30]:
response('thanks you')

u'My pleasure'